# AgentForce SDK - Advanced Integrations

This notebook demonstrates how to integrate the AgentForce SDK with popular AI frameworks:

* **LangChain**: For creating flexible language model chains and tools
* **LangGraph**: For building complex agent workflows with state management
* **LlamaIndex**: For data ingestion and RAG applications

## Prerequisites

First, let's install the necessary packages:

In [2]:
# Install the AgentForce SDK and required packages
!pip install --force-reinstall ../../dist/agentforce_sdk-0.1.4-py3-none-any.whl
!pip install "langchain>=0.1.12" "langchain-core>=0.1.31" langgraph==0.0.32 llama-index==0.10.5 openai

Processing /Users/ppadmanabhan/Documents/agent-sdk/dist/agentforce_sdk-0.1.4-py3-none-any.whl
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached simple_salesforce-1.12.6-py2.py3-none-any.whl.metadata (29 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached aiohttp-3.11.14-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached pytest_cov-6.0.0-py3-none-any.whl.metadata (27 kB)
  Using cached black-25.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (81 kB)
  Using cached flake8-7.1.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached isort-6.0.1-py3-none-any.whl.metadata (11 kB)
  Using cached mypy-1.15.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Us

## Setup

Let's import the necessary modules and set up our clients:

In [5]:
import os
from agent_sdk import Agentforce
#from agent_sdk.agent_utils import AgentUtils
from agent_sdk.models import Agent, Topic, Action, Input, Output
from agent_sdk.core.auth import BasicAuth

# LangChain imports
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
#from langchain_openai import ChatOpenAI

# LangGraph imports
from langgraph.graph import END, StateGraph
import langgraph.prebuilt as prebuilt

# LlamaIndex imports
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response_synthesizers import get_response_synthesizer

# Set API keys (replace with your own)
os.environ["OPENAI_API_KEY"] = "sk-JmEJUDl5gnv96APYIbAWUgooNmOQFK19QYSkQfM7i7T3BlbkFJfxQKlF_E6wrSm2GFqRLY9461a5mkvjgKGxM_5S1aYA"

# Initialize AgentForce
sf_username = "epic.d56b75fa672b@orgfarm.com"
sf_password = "orgfarm1234"
auth = BasicAuth(username=sf_username, password=sf_password)
agentforce = Agentforce(auth=auth)

2025-03-26 15:16:25 - agent_sdk.core.agent - INFO - Initializing Agentforce SDK
2025-03-26 15:16:25 - agent_sdk.core.agent - INFO - Running in development environment


## Part 1: AgentForce as a Tool in LangChain

In this example, we'll create a LangChain tool that uses the AgentForce SDK to interact with an agent.

In [6]:
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import Tool

# Define a tool for interacting with the AgentForce agent
@tool
def query_agent(query: str) -> str:
    """Use this to ask questions to the OrderManagementAgent in Salesforce."""
    agent_name = "OrderManagementAgent"  # Replace with your agent name
    response = agentforce.send_message(
        agent_name=agent_name,
        user_message=query
    )
    return response['agent_response']

# Create a LangChain agent with our custom tool
tools = [query_agent]
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant with access to a Salesforce agent for order management. "
              "Use the tools available to help answer questions about orders."),
    ("human", "{input}"),
    ("user", "{agent_scratchpad}")
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

NameError: name 'ChatOpenAI' is not defined

In [ ]:
# Test the LangChain agent
agent_executor.invoke({"input": "I'm looking for information about my order #12345. Can you help me?"})

## Part 2: Building a Customer Support Workflow with LangGraph

Here we'll create a more complex workflow using LangGraph, which handles customer inquiries by routing them to different systems including AgentForce.

In [ ]:
from typing import List, Dict, TypedDict, Annotated, Literal
import json

# Define our state
class State(TypedDict):
    query: str
    category: str
    answer: str
    history: List[Dict]

# Function to categorize the customer query
def categorize(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Categorize the user query into one of these categories: 'order', 'payment', 'general', 'escalate'"),
        ("human", "{query}")
    ])
    chain = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
    category = chain.invoke({"query": state["query"]})
    return {**state, "category": category.lower().strip()}

# Function to handle order queries with AgentForce
def handle_order_query(state: State) -> State:
    agent_name = "OrderManagementAgent"  # Replace with your agent name
    response = agentforce.send_message(
        agent_name=agent_name,
        user_message=state["query"]
    )
    return {**state, "answer": response['agent_response']}

# Function to handle payment queries
def handle_payment_query(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a payment specialist. Answer the user's payment-related query."),
        ("human", "{query}")
    ])
    chain = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
    answer = chain.invoke({"query": state["query"]})
    return {**state, "answer": answer}

# Function to handle general queries
def handle_general_query(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a general customer support assistant. Answer the user's query."),
        ("human", "{query}")
    ])
    chain = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
    answer = chain.invoke({"query": state["query"]})
    return {**state, "answer": answer}

# Function to handle escalations
def handle_escalation(state: State) -> State:
    return {**state, "answer": "I'm escalating your issue to a human agent who will contact you shortly. Your reference number is ESC-" + str(hash(state["query"]) % 10000)}

# Function to decide next step based on category
def route_query(state: State) -> Literal["order", "payment", "general", "escalate"]:
    category = state["category"]
    if "order" in category:
        return "order"
    elif "payment" in category:
        return "payment"
    elif "escalate" in category:
        return "escalate"
    else:
        return "general"

# Building the graph
customer_support_graph = StateGraph(State)

# Add the nodes
customer_support_graph.add_node("categorize", categorize)
customer_support_graph.add_node("order", handle_order_query)
customer_support_graph.add_node("payment", handle_payment_query)
customer_support_graph.add_node("general", handle_general_query)
customer_support_graph.add_node("escalate", handle_escalation)

# Add edges
customer_support_graph.add_edge("categorize", route_query)
customer_support_graph.add_edge("order", END)
customer_support_graph.add_edge("payment", END)
customer_support_graph.add_edge("general", END)
customer_support_graph.add_edge("escalate", END)

# Set the entry point
customer_support_graph.set_entry_point("categorize")

# Compile the graph
customer_support_app = customer_support_graph.compile()

In [ ]:
# Test the LangGraph workflow
result = customer_support_app.invoke({
    "query": "I need to check the status of my order #54321",
    "category": "",
    "answer": "",
    "history": []
})

print("Query Category:", result["category"])
print("\nResponse:")
print(result["answer"])

## Part 3: Enhancing AgentForce with RAG using LlamaIndex

Now we'll demonstrate how to use LlamaIndex to create a knowledge base for an AgentForce agent, enabling it to access custom data.

In [ ]:
# Create a simple knowledge base folder and document
import os
os.makedirs("knowledge_base", exist_ok=True)

# Create a sample product catalog document
with open("knowledge_base/product_catalog.txt", "w") as f:
    f.write("""
Product Catalog - Spring 2025

Product ID: P001
Name: Premium Widget
Price: $99.99
Description: Our flagship widget with advanced features.
Stock: 250 units

Product ID: P002
Name: Economy Widget
Price: $49.99
Description: Affordable option with essential features.
Stock: 500 units

Product ID: P003
Name: Professional Gadget
Price: $149.99
Description: High-performance tool for professionals.
Stock: 100 units
""")

In [ ]:
# Load documents from the knowledge base
documents = SimpleDirectoryReader("knowledge_base").load_data()

# Create a vector index from the documents
index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = index.as_query_engine()

In [ ]:
# Create an enhanced AgentForce function that uses LlamaIndex for product info
def enhanced_agent_query(query):
    # First check if it's a product query
    if "product" in query.lower() or "item" in query.lower() or "catalog" in query.lower():
        # Use LlamaIndex to answer product-related questions
        result = query_engine.query(query)
        return str(result)
    else:
        # Use AgentForce for other queries
        agent_name = "OrderManagementAgent"  # Replace with your agent name
        response = agentforce.send_message(
            agent_name=agent_name,
            user_message=query
        )
        return response['agent_response']

# Test the enhanced query function
print("Product Query Example:")
print(enhanced_agent_query("What is the price of the Premium Widget?"))
print("\nOrder Query Example:")
print(enhanced_agent_query("What's the status of order #12345?"))

## Part 4: Building a Hybrid Agent System

Now let's put it all together to create a hybrid system that combines LangChain, LangGraph, and LlamaIndex with AgentForce.

In [ ]:
# Create a tool for our LangChain agent that uses the enhanced function
@tool
def hybrid_query(query: str) -> str:
    """Query the hybrid system that combines product knowledge and order management."""
    return enhanced_agent_query(query)

# Create a LangChain agent with our hybrid tool
hybrid_tools = [hybrid_query]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a comprehensive customer service assistant that can help with product information "
              "and order management. Use the tools available to provide the best assistance."),
    ("human", "{input}"),
    ("user", "{agent_scratchpad}")
])

hybrid_agent = create_openai_tools_agent(llm, hybrid_tools, prompt)
hybrid_executor = AgentExecutor(agent=hybrid_agent, tools=hybrid_tools, verbose=True)

In [ ]:
# Test the hybrid system with a product query
hybrid_executor.invoke({"input": "What products do you have in stock and how much do they cost?"})

In [ ]:
# Test the hybrid system with an order query
hybrid_executor.invoke({"input": "I'd like to check the status of my order #54321 and also learn about the Professional Gadget."})

## Part 5: Creating an Integrated AgentForce Agent

Finally, let's demonstrate how to create and deploy an AgentForce agent that can leverage these integrations.

In [ ]:
# Define an action that will use our LlamaIndex knowledge base
product_lookup_action = Action(
    name="lookupProductInfo",
    description="Look up information about products in the catalog",
    inputs=[
        Input(
            name="productQuery",
            description="Query about a product",
            data_type="string"
        )
    ],
    outputs=[
        Output(
            name="productInfo",
            description="Information about the requested product",
            data_type="string"
        )
    ]
)

# Define a topic for product information
product_topic = Topic(
    name="Product Information",
    description="Provides information about products in the catalog",
    scope="public",
    instructions=[
        "When a user asks about a product, use the product lookup action to find information",
        "Provide details such as price, description, and availability"
    ],
    actions=[product_lookup_action]
)

# Define our integrated agent
integrated_agent = Agent(
    name="IntegratedShopAgent",
    description="A comprehensive shopping assistant that helps with product information and order management",
    agent_type="External",
    agent_template_type="EinsteinServiceAgent",
    company_name="Example Corp",
    sample_utterances=[
        "What products do you have available?",
        "I'd like to check the status of my order",
        "Tell me about the Premium Widget"
    ],
    topics=[product_topic]
)

# This is where you would deploy the agent to Salesforce
# result = agentforce.create(integrated_agent)
# print(f"Agent created successfully: {result}")

# Instead of deploying, let's just print the agent configuration
print(integrated_agent.to_json())

## Conclusion

In this notebook, we've demonstrated multiple ways to integrate the AgentForce SDK with modern AI frameworks:

1. Using AgentForce as a tool in LangChain
2. Building a customer support workflow with LangGraph that incorporates AgentForce
3. Enhancing AgentForce with RAG capabilities using LlamaIndex
4. Creating a hybrid system that combines all three frameworks
5. Designing an integrated AgentForce agent that leverages these capabilities

These integrations enable powerful capabilities such as:

* Smart routing of customer inquiries
* Access to external knowledge bases
* Multi-agent coordination
* Stateful conversations

To implement this in a production environment, you would need to:

1. Deploy the integrated AgentForce agent to Salesforce
2. Set up a service to handle the LlamaIndex knowledge base
3. Configure webhooks to connect external systems with the AgentForce agent
4. Implement authentication and security measures

For more information, refer to the documentation for each framework.